# JaBot: chat bot para presentar mi CV

A continuación, describiré los pasos para la construcción de un chatBot cuya función será contestar preguntas sobre mi CV.

Cargo las librerias que utilizo:


In [1]:
#!pip install spacy
#!python -m spacy download es_core_news_sm

In [24]:
import json
import spacy

## Obtención de datos

Los datos que voy a utilizar se han construido de forma manual. He reunido una serie de preguntas o frases que mi robot podría recibir, y las he clasificado en función de una intención, por ejemplo, la etiqueta'Greetings' reunirá todas las cuestiones relacionadas con saludos. También se reunirán un listado de respuestas para cada intención, que se elegirán de forma aleatoria (idea: utilizar un algoritmo de similitud, como coseno, jaccard) y el bot la utilizará para contestar. Para estructurar esta información utilizaré el formato Json.

Cargo los datos

In [34]:
def cargar_json(path):
    with open(path, encoding='utf-8') as file:
        return json.load(file)

data = cargar_json('../testFiles/intentsCV.json')
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

# limpieza de datos

1. Limpiar simbolos de puntuación: sustituir por espacios o borrarlo
2. lematizar
3. Borrar stop words

In [35]:
import unicodedata
nlp = spacy.load("es_core_news_sm")

#tokenizacion
def tokenizer(document):
    return [token for token in nlp(document)]

def clean_lemma(document,tokenize=True):
    if tokenize:
        return [remove_accented_chars(token.lemma_) for token in tokenizer(document) if token.is_alpha or token.is_digit]
    else:
        return ' '.join([remove_accented_chars(token.lemma_) for token in tokenizer(document) if token.is_alpha or token.is_digit])

def tokenizer_list(document_list):
    return [tokenizer(document) for document in document_list]

def clean_lemma_list(document_list,tokenize=True):
    return [clean_lemma(remove_accented_chars(document),tokenize) for document in document_list]

def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

In [36]:
sentences_list = clean_lemma_list(training_sentences,tokenize=False)
sentences_list[:2]


['haber alguien ahi alli', 'hey']

# Usar sklearn para convertir el texto a variables numéricas


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [38]:
cv = CountVectorizer(binary = True,
                    ngram_range=(1,3))

X_train = cv.fit_transform(sentences_list)

X_train.shape, len(training_labels)
lbl_encoder = LabelEncoder()

y_test = lbl_encoder.fit_transform(training_labels)

In [39]:
from sklearn.model_selection import GridSearchCV

def train_model(model,train,target):
    mod = model

    params = {'C':[0.01,0.05,0.25,0.5,1]}

    grid = GridSearchCV(mod,params,cv=5)
    grid.fit(train,target)

    return grid.best_estimator_

In [40]:
from sklearn.svm import LinearSVC

svm = LinearSVC()

best_svm= train_model(svm,X_train,y_test)

import pickle
with open('svm.pickle', 'wb') as rick:
    pickle.dump(best_svm, rick, protocol=pickle.HIGHEST_PROTOCOL)

C:\Users\Javier\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [43]:
import pickle
import random
import numpy as np

data = cargar_json('../testFiles/intentsCV.json')

with open('svm.pickle', 'rb') as handle:
    best_svm = pickle.load(handle)

while True:
    print("User: ", end="")
    inp = input()
    if inp.lower() == "quit":
        break
    input_text =  clean_lemma_list([inp],tokenize=False)
    #print(input_text)
    inp_code=cv.transform(input_text)
    resp = best_svm.predict(inp_code)
    
    tag = lbl_encoder.inverse_transform(resp)

    print(tag)

    for i in data['intents']:
        if i['tag'] == tag:
            print("ChatBot:", np.random.choice(i['responses']))

User: ['skills']
ChatBot: Sí, tengo bastante experiencia
User: 